In [1]:
# Import the libraries

import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import random
from pandas.io.json import json_normalize
import os

## Authorization Flow

In [2]:
# Declare the credentials

CLIENT_ID = # id do client da api
CLIENT_SECRET = # token do cilent da api
redirect_uri='http://localhost:7777/callback'
username = # seu username no spotify - Não é seu nome! Isso está disponível na parte Conta na platforma

# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

## Extract User's Top Medium Term Songs

In [3]:
directory = 'UserSongs'

In [4]:
all_songs = []

user_id = 0
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        user_songs = pd.read_csv(os.path.join(directory, filename))
        user_songs['user'] = user_id
        user_id += 1
        all_songs.append(user_songs)
        
all_songs = pd.concat(all_songs)
all_songs.reset_index(drop=True,inplace=True)
all_songs

,Unnamed: 0,artist,artist_uri,song,song_uri,duration_ms,explicit,album,popularity,user
0,0,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Science Fiction Double Feature (Glee Cast Vers...,spotify:track:4i3CKKl6S3fNkyGsgtJAss,267986,False,"Glee: The Music, The Rocky Horror Glee Show",42,0
1,1,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Sweet Transvestite (Glee Cast Version),spotify:track:5Uaz86QcbGZgTyS5sh6TmM,179920,False,"Glee: The Music, The Rocky Horror Glee Show",47,0
2,2,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Whatever Happened To Saturday Night? (Glee Cas...,spotify:track:3wZD98XkyFfkB2jHIORDnY,184400,False,"Glee: The Music, The Rocky Horror Glee Show",43,0
3,3,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Touch A Touch A Touch A Touch Me (Glee Cast Ve...,spotify:track:4QhUwFhOCM0qpORDogzvDs,149626,False,"Glee: The Music, The Rocky Horror Glee Show",49,0
4,4,Glee Cast,spotify:artist:0SCbttzoZTnLFebDYmAWCm,Time Warp (Glee Cast Version),spotify:track:1onMcvjW2nZ2dHtbJn2cN7,193560,False,"Glee: The Music, The Rocky Horror Glee Show",59,0
...,...,...,...,...,...,...,...,...,...,...
95,45,They Might Be Giants,spotify:artist:6zB02lwP6L6ZH32nggQiJT,Istanbul (Not Constantinople),spotify:track:63vL5oxWrlvaJ0ayNaQnbX,153813,False,Flood,67,1
96,46,O Grilo,spotify:artist:22KEpOwThQ5q1DGochfayO,Serenata Existencialista,spotify:track:1tIJba7S9uHsNml4BqZrah,163209,False,HerÃ³i do Futuro,47,1
97,47,Supercombo,spotify:artist:73HkjgziMO6I83vFOS8mo1,Todo Dia Ã‰ Dia de Comemorar,spotify:track:2TCAdLDCBwiNU7BdGcX8NE,213103,False,Todo Dia Ã‰ Dia de Comemorar,41,1
98,48,Selvagens Ã Procura de Lei,spotify:artist:093ybF4c81CndQe6qWtc8S,Gostar SÃ³ Dela,spotify:track:0FJCzNONkZwJurGiQYYmoH,187468,False,Gostar SÃ³ Dela,35,1


## Extract Users' Top 50 Tracks' Audio Features

In [5]:
audio_feat = []
for song in all_songs['song_uri']:
    row = pd.DataFrame(sp.audio_features(tracks=[song]))
    audio_feat.append(row)
audio_feat = pd.concat(audio_feat)

audio_feat.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.428,0.559,0,-7.052,1,0.0246,0.2820,0.000000,0.382,0.361,90.198,audio_features,4i3CKKl6S3fNkyGsgtJAss,spotify:track:4i3CKKl6S3fNkyGsgtJAss,https://api.spotify.com/v1/tracks/4i3CKKl6S3fN...,https://api.spotify.com/v1/audio-analysis/4i3C...,267987,4
0,0.736,0.571,4,-3.942,1,0.0457,0.4510,0.000618,0.223,0.554,107.788,audio_features,5Uaz86QcbGZgTyS5sh6TmM,spotify:track:5Uaz86QcbGZgTyS5sh6TmM,https://api.spotify.com/v1/tracks/5Uaz86QcbGZg...,https://api.spotify.com/v1/audio-analysis/5Uaz...,179920,4
0,0.566,0.913,3,-4.607,1,0.0577,0.1180,0.000005,0.194,0.961,164.982,audio_features,3wZD98XkyFfkB2jHIORDnY,spotify:track:3wZD98XkyFfkB2jHIORDnY,https://api.spotify.com/v1/tracks/3wZD98XkyFfk...,https://api.spotify.com/v1/audio-analysis/3wZD...,184400,4
0,0.309,0.789,10,-6.232,0,0.0457,0.2610,0.000068,0.163,0.606,170.713,audio_features,4QhUwFhOCM0qpORDogzvDs,spotify:track:4QhUwFhOCM0qpORDogzvDs,https://api.spotify.com/v1/tracks/4QhUwFhOCM0q...,https://api.spotify.com/v1/audio-analysis/4QhU...,149627,4
0,0.395,0.817,11,-5.757,0,0.1250,0.0442,0.000072,0.243,0.635,176.196,audio_features,1onMcvjW2nZ2dHtbJn2cN7,spotify:track:1onMcvjW2nZ2dHtbJn2cN7,https://api.spotify.com/v1/tracks/1onMcvjW2nZ2...,https://api.spotify.com/v1/audio-analysis/1onM...,193560,4


## Data Cleaning

In [6]:
# Drop unnecessary features

audio_feat.drop(['type','track_href','analysis_url','time_signature','duration_ms','uri','instrumentalness','liveness','loudness','key','mode'],1,inplace=True)
audio_feat.set_index('id',inplace=True)
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo
id,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.428,0.559,0.0246,0.2820,0.361,90.198
5Uaz86QcbGZgTyS5sh6TmM,0.736,0.571,0.0457,0.4510,0.554,107.788
3wZD98XkyFfkB2jHIORDnY,0.566,0.913,0.0577,0.1180,0.961,164.982
4QhUwFhOCM0qpORDogzvDs,0.309,0.789,0.0457,0.2610,0.606,170.713
1onMcvjW2nZ2dHtbJn2cN7,0.395,0.817,0.1250,0.0442,0.635,176.196


In [7]:
# Normalize tempo feature

columns = ['danceability','energy','speechiness','acousticness','valence','tempo']
scaler = MinMaxScaler()
scaler.fit(audio_feat[columns])
audio_feat[columns] = scaler.transform(audio_feat[columns])
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo
id,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.261417,0.532009,0.000000,0.288370,0.285377,0.136230
5Uaz86QcbGZgTyS5sh6TmM,0.746457,0.545254,0.051665,0.461913,0.512972,0.301606
3wZD98XkyFfkB2jHIORDnY,0.478740,0.922737,0.081048,0.119961,0.992925,0.839325
4QhUwFhOCM0qpORDogzvDs,0.074016,0.785872,0.051665,0.266805,0.574292,0.893206
1onMcvjW2nZ2dHtbJn2cN7,0.209449,0.816777,0.245837,0.044177,0.608491,0.944756


In [8]:
# Check out features summary

audio_feat.describe()

,danceability,energy,speechiness,acousticness,valence,tempo
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.491701,0.635155,0.083751,0.333909,0.520778,0.465225
std,0.204559,0.259895,0.148083,0.284920,0.272401,0.237688
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.388189,0.465232,0.020017,0.050569,0.288031,0.297525
50%,0.483465,0.713576,0.043095,0.263211,0.556604,0.480637
75%,0.624409,0.836645,0.080681,0.531484,0.742040,0.580732
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [9]:
#Making into A CSV

audio_feat.to_csv('AudioFeatures.csv')

## Building the Song Recommender

### K-Means Cluster Analysis

In [10]:
# Get 20 clusters from 100 songs

clusters = user_id*10
kmeans = KMeans(n_clusters=clusters)
kmeans.fit(audio_feat)
len(kmeans.labels_)

100

In [11]:
from sklearn.decomposition import PCA 

pca = PCA(3) 
pca.fit(audio_feat) 
  
pca_data = pd.DataFrame(pca.transform(audio_feat)) 
  
print(pca_data.head())

          0         1         2
0  0.183691  0.173353  0.381670
1  0.166068 -0.248829  0.097882
2 -0.576117 -0.067607 -0.370107
3 -0.222279  0.403955 -0.345443
4 -0.406052  0.381633 -0.353158


In [14]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(audio_feat)
y_kmeans = kmeans.fit_predict(scaled)
y_kmeans

array([17,  7,  3,  6,  6,  3, 15, 11,  7,  5, 11, 13,  2,  9, 11, 11,  1,
       13, 12, 10, 11, 13, 12,  2,  7,  7, 11,  3, 19,  3, 12, 10,  2,  7,
       18, 19, 12,  7, 13,  3, 11, 19, 13, 12,  5,  9,  0,  7, 13, 11,  4,
        1,  4,  9,  6, 18, 18,  3, 18,  4,  4, 15, 14,  3,  4, 16,  4, 15,
       16,  2, 19,  1, 10,  5,  0, 17, 12,  8, 19, 19, 10, 10, 15, 17, 18,
        9,  4, 19,  3, 18, 19,  0, 18,  4,  9, 13, 19, 18, 11, 15])

In [15]:
# Updating dataframe with assigned clusters 

audio_feat['cluster'] = y_kmeans
audio_feat['artist'] = all_songs.artist.tolist()
audio_feat['title'] = all_songs.song.tolist()
audio_feat['user'] = all_songs.user.tolist()
audio_feat.head()

,danceability,energy,speechiness,acousticness,valence,tempo,cluster,artist,title,user
id,,,,,,,,,,
4i3CKKl6S3fNkyGsgtJAss,0.261417,0.532009,0.000000,0.288370,0.285377,0.136230,17,Glee Cast,Science Fiction Double Feature (Glee Cast Vers...,0
5Uaz86QcbGZgTyS5sh6TmM,0.746457,0.545254,0.051665,0.461913,0.512972,0.301606,7,Glee Cast,Sweet Transvestite (Glee Cast Version),0
3wZD98XkyFfkB2jHIORDnY,0.478740,0.922737,0.081048,0.119961,0.992925,0.839325,3,Glee Cast,Whatever Happened To Saturday Night? (Glee Cas...,0
4QhUwFhOCM0qpORDogzvDs,0.074016,0.785872,0.051665,0.266805,0.574292,0.893206,6,Glee Cast,Touch A Touch A Touch A Touch Me (Glee Cast Ve...,0
1onMcvjW2nZ2dHtbJn2cN7,0.209449,0.816777,0.245837,0.044177,0.608491,0.944756,6,Glee Cast,Time Warp (Glee Cast Version),0


In [18]:
# Removing clusters that only have one song in them

delete_clusters = []
cluster = 0
while cluster < (len(audio_feat.cluster.unique())-1):
    if audio_feat.groupby('cluster').count().loc[cluster].danceability == 1:
        delete_clusters.append(cluster)
    cluster+=1
    
delete_clusters

[8, 14]

In [19]:
audio_feat.reset_index(inplace=True)

In [20]:
i = 0
while i < (len(audio_feat.cluster.unique())-1):
    if audio_feat.loc[[i]].cluster.tolist()[0] in delete_clusters:
        audio_feat.drop(i,0,inplace=True)
    i+=1

In [21]:
audio_feat.set_index('id',inplace=True)

In [22]:
# Create list of lists of song ids to put into recommendation function

i=0
list_of_recs = [0]*len(audio_feat.groupby('cluster').count())
while i<len(audio_feat.groupby('cluster').count()):
    list_of_recs[i] = audio_feat.loc[audio_feat['cluster'] == i].index.tolist()
    i+=1

list_of_recs = [ele for ele in list_of_recs if ele != []] 
len(list_of_recs)

20

In [23]:
# Adjust list for clusters so that each cluster has a maximum of 5 seed songs

j = 0
adj_list_of_recs = [0]*len(list_of_recs)
while j<len(list_of_recs):
    if 0 < len(list_of_recs[j]) < 6:
        adj_list_of_recs[j] = list_of_recs[j]
    elif len(list_of_recs[j]) > 5:
        adj_list_of_recs[j] = random.sample(list_of_recs[j], 5)
    j += 1

len(adj_list_of_recs)

20

In [24]:
#Getting 1 recommended song from each cluster with less than 4 songs, 2 recommended songs from each cluster with 4-5 songs

k = 0
list_of_recommendations = [0]*len(list_of_recs)
while k < len(list_of_recs):
    if len(adj_list_of_recs[k]) < 4:
        list_of_recommendations[k] = sp.recommendations(seed_tracks=adj_list_of_recs[k],limit=1)
    else:
        list_of_recommendations[k] = sp.recommendations(seed_tracks=adj_list_of_recs[k],limit=2)
    k += 1
    
pd.json_normalize(list_of_recommendations[15], record_path='tracks').id

0    1REX2q4Fverp8zH7KuSeZC
1    6bsSMbRhnPdAg7ZR8xXE7r
Name: id, dtype: object

In [25]:
list_of_recommendations_converted = [0]*len(list_of_recs)

l = 0
while l < len(list_of_recs):
    list_of_recommendations_converted.append(pd.json_normalize(list_of_recommendations[l], record_path='tracks').id.tolist())
    l += 1

no_integers = [x for x in list_of_recommendations_converted if not isinstance(x, int)]
list_of_recommendations_converted = [item for elem in no_integers for item in elem]

len(list_of_recommendations_converted)

32

In [27]:
# Authorization flow

scope = "playlist-modify-public"
token = util.prompt_for_user_token(username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [28]:
# Create new playlist and insert it straight to user's library

def create_playlist(sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)

In [29]:
create_playlist(sp, username, 'Spotify Discover Together', 'Choose a friend to discover brand new music with. We create an adventurous playlist curated to both of your tastes!')

In [30]:
# Fetch user's playlist library

def fetch_playlists(sp, username):
    """
    Returns the user's playlists.
    """
        
    id = []
    name = []
    num_tracks = []
    
    # Make the API request
    playlists = sp.user_playlists(username)
    for playlist in playlists['items']:
        id.append(playlist['id'])
        name.append(playlist['name'])
        num_tracks.append(playlist['tracks']['total'])

    # Create the final df   
    df_playlists = pd.DataFrame({"id":id, "name": name, "#tracks": num_tracks})
    return df_playlists

In [31]:
# Find the new playlist's id

fetch_playlists(sp,username).head(3)

,id,name,#tracks
0,3TSfB52P3BSXqXnWYlKD9T,Spotify Discover Together,0
1,1dLIGzCAAvrS73FGayI0zf,Loka de Pinga – Kika Boom,7
2,0A5FvyY8Bzu2OExvDUioer,Ana Vitória + brasileiras,60


In [32]:
extracted_id = fetch_playlists(sp,username).id[0]

In [33]:
# Finally, fill the new playlist with the recommended songs straight to the user's library!

sp.user_playlist_add_tracks(username, extracted_id, list_of_recommendations_converted, position=None)

{'snapshot_id': 'Myw4MjlmY2M3YjM1MmY1N2E2MTc4YTFjYjkwYTZkY2NhOTU5NzdmM2Fl'}